<a href="https://colab.research.google.com/github/ahsank/StockML/blob/main/StockML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q yahoo_fin

In [ ]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [ ]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [ ]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '9b', tickers, True)

In [ ]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
78,WOLF,2.88,0.95,1009.94,948.93,8.10,11.61,0.43,2.19,0.73,1063.04,1128.08,11.24,0.39,1.66,0.78,814.78,988.06,8.55,0.06,9.383721
55,PLUG,1.07,0.90,432.89,454.85,1.94,2.53,0.30,0.76,0.73,464.35,353.54,3.56,0.84,0.58,0.75,178.77,428.65,1.73,-0.11,8.666667
27,FOUR,1.74,0.93,643.19,541.87,86.90,93.54,0.08,0.98,0.82,885.52,713.64,95.92,0.10,0.89,0.63,561.25,484.94,76.55,-0.12,8.425000
64,SNOW,5.08,0.90,1542.56,2486.30,110.27,124.56,0.13,3.76,0.74,1876.85,2645.62,122.62,0.11,2.96,0.75,2496.45,3035.14,112.39,0.02,8.230769
5,BASE,0.66,0.92,168.61,226.44,13.67,14.75,0.08,0.45,0.70,169.12,207.37,15.84,0.16,0.35,0.71,152.37,222.19,12.02,-0.12,8.200000
24,ETSY,3.82,0.90,1360.03,1569.90,53.91,60.24,0.12,2.99,0.71,896.10,1653.78,62.75,0.16,1.99,0.76,1304.36,1688.69,53.47,-0.01,8.166667
37,JKS,1.46,0.90,291.10,596.04,19.74,21.86,0.11,0.93,0.62,408.23,557.43,21.99,0.11,0.58,0.76,521.00,688.75,17.41,-0.12,8.090909
10,CHGG,1.49,0.83,175.53,618.02,1.74,3.08,0.77,1.15,0.63,19.97,324.30,3.17,0.82,1.02,0.69,126.63,576.46,2.25,0.29,8.079221
29,GTLB,1.79,0.90,410.77,466.71,52.05,57.07,0.10,1.06,0.74,491.32,626.96,58.04,0.12,1.00,0.70,447.48,360.47,48.07,-0.08,8.000000
7,BMBL,0.88,0.85,145.98,337.06,6.09,7.10,0.17,0.63,0.70,179.81,284.16,8.21,0.35,0.63,0.79,271.97,393.76,5.39,-0.11,7.617647


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '7b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
46,XNTK,1.66,0.98,866.49,766.13,191.95,174.58,-0.09,0.97,0.83,970.66,756.01,194.19,0.01,1.19,0.67,781.89,570.50,169.05,-0.12,9.577778
34,XLB,0.58,0.98,375.82,220.82,94.07,91.60,-0.03,0.30,0.80,457.28,310.25,95.30,0.01,0.35,0.72,333.80,231.70,90.07,-0.04,9.133333
28,VNQ,0.57,0.96,344.73,341.10,97.79,104.88,0.07,0.28,0.80,435.53,300.05,100.57,0.03,0.30,0.70,384.78,211.47,99.11,0.01,9.028571
42,XLU,0.50,0.95,272.51,198.18,80.08,74.25,-0.07,0.24,0.84,351.21,227.77,76.26,-0.05,0.23,0.65,241.17,94.40,70.29,-0.12,8.785714
3,CIBR,0.45,0.94,211.67,140.39,59.30,53.80,-0.09,0.30,0.73,211.08,132.44,60.65,0.02,0.25,0.71,256.62,199.17,53.74,-0.09,8.733333
41,XLRE,0.31,0.97,201.02,137.73,44.68,45.86,0.03,0.16,0.81,188.71,121.49,45.74,0.02,0.16,0.63,139.81,118.56,43.89,-0.02,8.700000
31,VUG,2.24,0.95,1489.90,1009.49,380.89,357.59,-0.06,1.41,0.79,1139.53,772.55,382.84,0.01,1.51,0.70,1260.49,1097.45,346.09,-0.09,8.666667
19,MGK,2.11,0.93,1123.67,843.59,319.32,285.75,-0.11,1.07,0.83,1269.86,743.84,322.00,0.01,1.19,0.83,1576.50,1382.96,293.84,-0.08,8.663636
16,IWO,2.42,0.96,1099.42,1133.34,284.01,272.06,-0.04,1.20,0.76,1099.86,1089.75,289.19,0.02,1.20,0.74,1377.57,1218.43,269.48,-0.05,8.600000
2,ARKW,1.60,0.94,514.51,891.02,84.03,89.54,0.07,1.21,0.64,341.15,537.93,88.33,0.05,1.28,0.79,801.91,1050.43,69.49,-0.17,8.542857


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
lossfn = findata.Huber()
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DPSI', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombinedVola(tickers3, 'vols-8b', mod, True, loss=lossfn)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
25,GOGO,0.29,0.95,134.12,153.34,7.24,8.32,0.15,0.31,0.66,163.99,142.46,8.13,0.12,0.18,0.73,68.62,93.26,6.85,-0.05,5.254902
36,LAZR,0.46,0.90,85.33,216.86,0.91,1.37,0.51,0.74,0.49,-11.82,104.34,1.37,0.51,0.30,0.68,99.08,219.98,0.96,0.05,5.098039
34,JOBY,0.18,0.92,57.28,88.64,5.22,6.50,0.25,0.26,0.56,46.15,91.44,5.77,0.10,0.13,0.72,54.46,93.53,4.93,-0.05,5.091765
3,AMSC,0.50,0.90,164.02,190.06,22.98,30.19,0.31,0.50,0.56,191.42,161.51,30.71,0.34,0.35,0.72,116.30,130.84,16.25,-0.29,4.971537
27,GRPN,0.83,0.89,191.61,307.57,9.96,13.16,0.32,0.94,0.70,188.22,242.38,13.57,0.36,0.62,0.67,103.50,269.32,9.34,-0.06,4.891544
74,YEXT,0.24,0.90,61.14,106.87,6.72,7.78,0.16,0.33,0.59,47.03,68.77,7.90,0.18,0.17,0.74,49.56,85.68,6.19,-0.08,4.669118
20,CXM,0.28,0.94,82.46,135.85,7.73,8.18,0.06,0.32,0.64,67.17,77.48,9.33,0.21,0.21,0.66,48.49,103.06,7.45,-0.04,4.529412
31,INDI,0.26,0.90,86.63,108.92,4.17,4.71,0.13,0.21,0.64,67.07,77.71,4.62,0.11,0.14,0.74,70.39,93.01,3.77,-0.09,4.524887
18,CLOV,0.29,0.83,49.09,108.79,2.82,4.06,0.44,0.44,0.45,48.69,105.73,6.02,1.14,0.10,0.64,28.87,80.85,2.07,-0.26,4.495989
64,SLE,2.41,0.76,521.21,863.30,0.65,1.21,0.88,2.46,0.52,915.71,1119.32,3.51,4.44,1.27,0.75,622.75,677.78,0.97,0.50,4.197861


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombinedVola(tickers4, 'mcap-8b', mod4, False)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
6,BA,3.72,0.96,1195.28,1609.71,156.32,176.98,0.13,2.57,0.65,985.34,1151.74,168.69,0.08,1.89,0.72,835.87,1119.20,130.00,-0.17,6.549451
2,ADBE,8.44,0.95,4846.85,3562.30,515.48,563.26,0.09,5.12,0.73,3141.35,2700.64,543.30,0.05,5.33,0.67,2845.61,2495.04,501.29,-0.03,6.230159
11,CRM,3.61,0.95,1602.30,1592.33,276.64,255.47,-0.08,2.45,0.74,1298.56,943.42,278.62,0.01,2.48,0.71,1092.29,1277.39,242.16,-0.12,6.160714
33,QCOM,2.16,0.94,1166.71,877.31,170.13,155.44,-0.09,1.63,0.72,856.04,769.76,184.31,0.08,1.32,0.73,901.70,834.28,157.37,-0.08,6.047619
36,TSLA,6.45,0.93,3725.14,2384.86,260.46,231.35,-0.11,4.27,0.68,2785.49,2422.26,253.77,-0.03,4.01,0.72,2389.93,2341.48,256.33,-0.02,6.006494
3,AMD,2.71,0.95,1301.09,965.56,164.35,154.18,-0.06,2.10,0.64,952.01,575.15,177.13,0.08,1.52,0.78,1282.34,919.43,135.89,-0.17,5.952381
22,IBM,2.04,0.93,744.11,505.74,220.84,197.88,-0.10,1.63,0.72,865.84,211.19,213.71,-0.03,1.08,0.61,489.12,184.94,199.93,-0.09,5.942857
10,CDNS,3.72,0.94,1512.21,1033.02,274.13,289.49,0.06,2.35,0.77,1415.51,881.66,291.63,0.06,1.99,0.63,1066.94,662.10,255.19,-0.07,5.714286
31,ORCL,1.85,0.91,712.35,399.71,168.74,150.31,-0.11,1.50,0.78,886.35,400.00,171.42,0.02,1.06,0.69,508.97,164.85,146.58,-0.13,5.681818
4,AMZN,2.47,0.93,1120.41,976.24,187.97,174.02,-0.07,1.49,0.78,1007.18,828.11,186.03,-0.01,1.39,0.70,604.04,1023.01,173.70,-0.08,5.642857


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '10a', tickers5, True)

In [13]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
23,FQVLF,0.65,0.93,229.00,278.26,13.78,16.98,0.23,0.48,0.72,160.62,214.45,16.38,0.19,0.39,0.79,229.23,299.45,13.30,-0.03,6.034161
56,STLA,0.34,0.83,97.20,93.11,13.32,17.96,0.35,0.25,0.72,107.37,88.15,17.32,0.30,0.18,0.73,112.17,99.90,13.80,0.04,4.957143
45,PAX,0.20,0.92,58.88,73.44,11.00,11.91,0.08,0.14,0.72,49.86,82.27,11.63,0.06,0.08,0.77,87.67,84.13,10.69,-0.03,4.571429
33,MGA,1.04,0.91,401.52,413.14,41.08,44.83,0.09,0.64,0.78,343.91,437.20,45.45,0.11,0.78,0.73,367.85,448.95,39.32,-0.04,4.500000
52,ORAN,0.09,0.94,45.35,31.64,10.74,11.27,0.05,0.06,0.71,34.71,23.72,11.64,0.08,0.05,0.78,41.37,40.40,10.91,0.02,4.314286
51,SFTBY,0.63,0.93,183.84,249.56,29.86,31.76,0.06,0.39,0.71,193.07,144.26,31.44,0.05,0.36,0.78,180.21,175.92,25.47,-0.15,4.309524
3,BAK,0.35,0.93,105.49,160.14,7.38,7.78,0.05,0.31,0.72,106.90,113.27,7.49,0.02,0.22,0.82,132.10,161.49,6.43,-0.13,3.842857
4,BASFY,0.19,0.94,65.83,88.35,13.24,13.58,0.03,0.11,0.72,44.06,67.23,13.35,0.01,0.11,0.72,59.90,70.84,12.15,-0.08,2.714286
37,LVMUY,2.55,0.92,937.19,748.10,146.70,152.10,0.04,1.72,0.64,578.77,662.60,152.19,0.04,1.28,0.71,746.66,548.61,135.44,-0.08,2.571429
25,GLOB,5.64,0.88,1722.62,1827.03,193.45,205.48,0.06,3.35,0.68,1251.72,1673.24,202.87,0.05,2.56,0.82,1929.21,2459.61,192.50,-0.00,2.285714


In [ ]:
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '8a', tickers6, False)

In [15]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
3,ECH,0.29,0.96,138.96,155.62,26.52,24.42,-0.08,0.19,0.79,129.13,122.16,26.68,0.01,0.21,0.66,96.61,100.67,24.68,-0.07,9.100000
15,EWZ,0.30,0.95,158.10,157.80,29.41,27.16,-0.08,0.19,0.76,182.47,184.27,30.12,0.02,0.17,0.74,184.15,142.99,26.80,-0.09,8.875000
17,ILF,0.28,0.94,135.46,94.45,26.05,23.56,-0.10,0.14,0.76,141.48,101.59,25.34,-0.03,0.17,0.65,87.15,51.80,22.90,-0.12,8.800000
2,FLN,0.17,0.93,72.74,76.31,17.90,16.68,-0.07,0.11,0.81,96.35,85.14,17.60,-0.02,0.08,0.74,98.09,83.73,16.53,-0.08,8.300000
12,EWT,0.52,0.88,171.52,156.65,53.78,47.58,-0.12,0.35,0.75,173.06,133.64,53.19,-0.01,0.47,0.70,169.77,153.10,46.28,-0.14,7.800000
14,EWY,0.72,0.92,224.74,332.33,62.70,59.87,-0.05,0.45,0.71,191.58,253.62,65.22,0.04,0.56,0.68,184.21,271.46,58.69,-0.06,7.600000
4,EPHE,0.32,0.92,116.84,104.13,29.31,27.83,-0.05,0.22,0.70,66.90,92.67,28.71,-0.02,0.23,0.66,75.92,69.74,26.66,-0.09,7.600000
21,TUR,0.43,0.91,234.84,121.72,34.40,32.66,-0.05,0.37,0.76,184.25,136.48,35.89,0.04,0.36,0.72,145.69,167.57,34.09,-0.01,7.300000
8,EWJ,0.58,0.91,200.58,161.05,71.88,67.98,-0.05,0.39,0.73,193.03,128.81,71.47,-0.01,0.46,0.67,195.14,172.24,66.10,-0.08,7.300000
1,EWD,0.39,0.92,154.97,174.09,42.23,40.38,-0.04,0.21,0.80,212.26,145.58,42.53,0.01,0.27,0.63,133.39,142.93,39.04,-0.08,7.200000


In [ ]:
findata.EPOCHS=200
review = [ 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
df7 = run_model('val', '9a', tickers7, True)

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
2,ALB,4.76,0.96,2196.83,2421.23,76.90,85.28,0.11,3.17,0.79,2184.78,2524.58,97.33,0.27,2.25,0.75,1785.61,2370.41,79.63,0.04,9.236364
37,FCX,0.80,0.97,352.36,295.53,40.00,42.37,0.06,0.48,0.78,438.95,294.27,44.88,0.12,0.39,0.76,427.02,333.85,37.27,-0.07,9.200000
12,BGFV,0.40,0.92,136.50,151.87,2.12,3.13,0.48,0.38,0.70,274.10,250.96,2.51,0.18,0.23,0.74,120.76,181.93,1.68,-0.21,9.033333
0,AAL,0.34,0.93,170.91,129.51,10.80,13.48,0.25,0.18,0.76,190.35,182.11,12.85,0.19,0.17,0.80,159.30,165.57,10.48,-0.03,9.020000
73,RH,8.82,0.91,3008.27,4220.16,248.09,330.04,0.33,5.44,0.66,2244.51,3609.01,305.48,0.23,4.51,0.72,2368.56,2964.78,241.69,-0.03,8.827273
33,DVN,0.94,0.93,428.39,307.30,41.34,47.04,0.14,0.62,0.80,363.72,225.09,43.32,0.05,0.59,0.65,376.52,175.96,39.67,-0.04,8.800000
16,CC,0.57,0.91,259.27,262.64,17.47,21.18,0.21,0.30,0.79,299.60,352.91,21.96,0.26,0.33,0.71,250.22,255.04,16.58,-0.05,8.671429
67,PETS,0.51,0.89,80.74,173.38,3.29,4.32,0.31,0.38,0.61,203.31,271.22,3.93,0.20,0.25,0.87,171.96,244.01,3.13,-0.05,8.545161
28,COP,1.51,0.93,771.70,340.65,106.02,113.74,0.07,1.09,0.76,748.54,335.09,116.73,0.10,0.90,0.73,701.58,449.97,98.38,-0.07,8.300000
30,DAL,0.60,0.94,344.47,282.16,42.03,43.92,0.05,0.35,0.72,281.31,312.72,42.72,0.02,0.29,0.73,279.29,306.18,38.88,-0.07,8.200000


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXB', 'FXE', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '9a', tickers8, True)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
9,TLT,1.21,0.96,261.01,449.66,100.41,94.09,-0.06,1.16,0.65,65.41,273.97,103.12,0.03,0.85,0.82,211.38,460.47,99.47,-0.01,8.933333
6,SLV,0.31,0.96,109.71,107.74,28.02,26.97,-0.04,0.26,0.70,113.63,103.20,27.95,-0.00,0.17,0.79,138.19,148.88,25.72,-0.08,8.600000
7,SOYB,0.30,0.93,102.31,76.26,22.12,21.05,-0.05,0.24,0.66,71.13,50.40,22.16,0.00,0.24,0.76,117.17,103.71,20.10,-0.09,7.900000
3,GLD,1.40,0.93,633.32,436.18,238.68,229.37,-0.04,1.44,0.75,633.73,328.44,237.72,-0.00,0.92,0.63,352.18,80.26,219.89,-0.08,7.550000
13,WEAT,0.14,0.88,37.19,42.60,5.30,5.74,0.08,0.12,0.64,25.33,38.61,5.51,0.04,0.08,0.77,40.95,54.25,5.07,-0.04,7.300000
10,USO,1.30,0.93,461.72,273.47,69.84,71.08,0.02,1.14,0.66,272.74,169.49,73.89,0.06,0.89,0.65,397.22,244.27,67.95,-0.03,5.800000
2,FXY,0.55,0.90,99.28,195.93,65.60,63.70,-0.03,0.59,0.56,30.59,115.29,67.00,0.02,0.45,0.82,55.07,183.49,63.87,-0.03,5.666667
4,ISHG,0.36,0.89,81.00,141.94,73.42,74.88,0.02,0.31,0.69,64.59,89.53,74.17,0.01,0.34,0.77,63.42,114.14,72.36,-0.01,3.400000
1,FXE,0.40,0.93,150.27,135.34,102.28,103.29,0.01,0.46,0.67,103.73,119.98,103.95,0.02,0.34,0.77,127.11,170.51,102.45,0.00,2.300000
5,PDBC,0.17,0.91,58.59,31.67,13.05,13.15,0.01,0.18,0.62,29.21,18.25,13.49,0.03,0.10,0.67,53.52,37.39,13.02,-0.00,0.100000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '9a', tickers9, False)

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
11,MATIC-USD,0.05,0.88,7.13,12.81,0.42,0.56,0.33,0.03,0.67,8.82,13.35,1.02,1.41,0.01,0.73,8.93,11.70,0.37,-0.13,5.922078
8,ICP-USD,0.46,0.89,117.01,163.30,8.65,7.68,-0.11,0.41,0.65,140.78,175.97,9.19,0.06,0.19,0.73,112.44,179.51,7.52,-0.13,5.357143
14,TON-USD,0.10,0.88,19.87,30.47,1.00,1.13,0.12,0.11,0.70,59.28,72.70,1.31,0.30,0.03,0.77,24.77,43.49,0.91,-0.10,5.285714
6,ETH-USD,77.31,0.87,22744.09,23195.91,2429.74,2079.19,-0.14,59.94,0.67,13314.51,19795.80,2530.01,0.04,28.64,0.69,17327.94,12427.05,2102.62,-0.13,5.285714
16,XRP-USD,0.02,0.89,5.23,6.04,0.58,0.52,-0.10,0.01,0.66,4.99,5.42,0.58,0.00,0.00,0.79,5.62,5.97,0.44,-0.24,5.257143
15,TRX-USD,0.00,0.87,0.89,0.35,0.15,0.13,-0.13,0.00,0.78,0.99,0.53,0.15,0.02,0.00,0.56,0.52,0.32,0.13,-0.15,5.214286
4,DOGE-USD,0.00,0.85,0.88,1.40,0.11,0.09,-0.12,0.00,0.55,0.73,1.37,0.11,0.06,0.00,0.73,0.75,1.32,0.09,-0.16,4.821429
7,FIL-USD,0.51,0.83,90.37,147.48,3.71,3.27,-0.12,0.41,0.70,95.33,132.67,3.95,0.06,0.19,0.71,62.10,115.45,3.05,-0.18,4.511905
5,DOT-USD,0.45,0.85,104.00,159.30,4.42,4.04,-0.09,0.31,0.67,49.19,127.90,4.45,0.01,0.18,0.76,79.09,135.49,3.77,-0.15,4.404762
0,ADA-USD,0.02,0.88,5.86,7.25,0.36,0.34,-0.06,0.02,0.65,5.34,7.12,0.37,0.04,0.01,0.74,4.46,9.49,0.32,-0.10,4.285714


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
mcap-9b-adjclose-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
mcap-9b-high-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
mcap-9b-low-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras

sent 28,834,389 bytes  received 73 bytes  57,668,924.00 bytes/sec
total size is 86,477,638  speedup is 3.00
